In [ ]:
import numpy as np

**Kalman Filtering Equations**

$x_{k}$= Α$x_{k-1}$ + B$u_{k}$ + $w_{k-1}$

$y_{k}$= H$x_{k}$ + $v_{k}$

where


*   $x_{k}$ represents state of process at time step k
*   $y_{k}$ is the measurement recorded at time step k

*   $w_{k}$ ≈ N(0,Q) , Q process noise covariance

*   $v_{k}$ ≈ N(0,R) , R measurement noise covariance and indepedent to process noise




<br>




*   **Prediction**


1.   $X^{-}_{k}$ = $A_{k-1}$  $X_{k-1}$ + $B_{k}$  $U_{k}$

2.   $P^{-}_{k}$ = $A_{k-1}$$P_{k-1}$$A^{T}_{k-1}$ + $Q_{k-1}$


*   **Update**


1.   $V_{k}$ = $Y_{k}$ - $H_{k}$$X^{-}_{k}$

2.   $S_{k}$ = $H_{k}$$P^{-}_{k}$$H^{T}_{k}$ + $R_{k}$

3. $K_{k}$ = $P^{-}_{k}$$H^{T}_{k}$$S^{-1}_{k}$

4. $X_{k}$ = $X^{-}_{k}$ + $K_{k}$$V_{k}$

5. $P_{k}$ = $P^{-}_{k}$ - $K_{k}$$S_{k}$$K^{T}_{k}$

where



*   $X^{-}_{k}$ and $P^{-}_{k}$ are the predicted mean and covariance of the state,respectively at time step k before seeing the measurement

*   $X_{k}$ and $P_{k}$ are the estimated mean and covariance of the state,respectively at time step k after seeing the measurement

*   $Y_{k}$ is mean of the measurement on time step k

*   $V_{k}$ is the innovation or the measurement residual on time step k

*   $S_{k}$ is the measurement prediction covariance on time step k

*   $K_{k}$ is the filter gain, which tells us how much the predictions should be corrected on time step k











In [ ]:
def kf_predict(X,P,A,Q,B,U):
  X= np.dot(A,X) + np.dot(B,U)
  P= np.dot(A,np.dot(P,np.transpose(A))) + Q
  return (X,P)

In [ ]:
def kf_update(X,P,Y,H,R):
  IM= np.dot(H,X)
  IS= R+ np.dot(H,np.dot(P,np.transpose(H)))
  K= np.dot(P,np.dot(np.transpose(H),np.linalg.inv(IS)))
  X= X + np.dot(K,Y-IM)
  P= P - np.dot(K,np.dot(IS,np.transpose(K)))
  LH= gauss_pdf(Y,IM,IS)
  return (X,P,K,IM,IS,LH)

def gauss_pdf(X,M,S):
  if M.shape[1]==1:
    dx= X - np.tile(M,X.shape[1])
    E= 0.5 * np.sum(dx * np.dot(np.linalg.inv(S),dx),axis=0)
    E = E + 0.5 *M.shape[0] * np.log(2*np.pi) + 0.5* np.log(np.linalg.det(S))
    P= np.exp(-E)
  elif X.shape[1]==1:
    dx= np.tile(X,M.shape[1]) - M
    E= 0.5 * np.sum(dx * np.dot(np.linalg.inv(S),dx),axis=0)
    E = E + 0.5 *M.shape[0] * np.log(2*np.pi) + 0.5*np.log(np.linalg.det(S))
    P= np.exp(-E)
  else:
    dx= X - M
    E= 0.5 * np.sum(np.transpose(dx) * np.dot(np.linalg.inv(S),dx),axis=0)
    E = E + 0.5 *M.shape[0] * np.log(2*np.pi) + 0.5* np.log(np.linalg.det(S))
    P= np.exp(-E)
  return (P[0],E[0])

In [ ]:
#time step
dt=0.1

#Initialisation of state matrices
X= np.array([[0.0],[0.0],[0.1],[0.1]])
P=np.diag((0.01,0.01,0.01,0.01))
A= np.array([[1,0,dt,0],[0,1,0,dt],[0,0,1,0],[0,0,0,1]])
Q=np.eye(X.shape[0])
B= np.eye(X.shape[0])
U= np.zeros((X.shape[0],1))

#Initialisation of measurement matrices
Y=np.array([[X[0,0] + abs(np.random.normal(0,1))],[X[1,0]+ abs(np.random.normal(0,1))]])
H=np.array([[1,0,0,0],[0,1,0,0]])
R=np.eye(Y.shape[0])

num=50

for i in np.arange(0,num):
  (X,P)=kf_predict(X,P,A,Q,B,U)
  (X,P,K,IM,IS,LH)=kf_update(X,P,Y,H,R)
  Y= np.array([[X[0,0] + abs(0.1*np.random.normal(0,1))],[X[1,0]+ abs(0.1*np.random.normal(0,1))]])



array([[4.14813421],
       [4.36499441]])